In [56]:
import gymnasium as gym
import gymnasium_robotics

# PyTorch
import torch

import os

# from collections import deque
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

In [57]:
env_id = 'FrankaKitchen-v1'
task = 'kettle'
gym.register_envs(gymnasium_robotics)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_actions = 9

In [58]:
granularity = 2
transform_action_space = np.linspace(start=-1.0, stop=1.0, num=granularity)

def transform_action_from_int(action: int):
    if isinstance(action, list) :
        return action
    array_action = np.zeros(9)
    if action > 18 or action < 0:
        raise AssertionError("transform_action_from_int")
    if action == 18:
        return array_action
    which_action = action % 9
    singular_action = transform_action_space[action // 9]
    array_action[which_action] = singular_action
    return array_action

def transform_action_to_int(action) -> int:
    if isinstance(action, np.int64) :
        return action
    if sum(action) == 0.0:
        return 18
    for i in range(9):
        if action[i] != 0.0:
            value = action[i]
            closest_quantized = min(transform_action_space, key=lambda x:abs(x - value))
            closest_quantized_index = -1
            for j in range(len(transform_action_space)):
                if transform_action_space[j] == closest_quantized:
                    closest_quantized_index = j
            if closest_quantized_index == -1: raise AssertionError("transform_action_to_int index is -1")
            return i + 9 * closest_quantized_index
    raise AssertionError("transform_action_to_int shouldn't be here")

def flatten_observation(observation):
    if not isinstance(observation, dict):
        return observation
    achieved = observation['achieved_goal'][task].astype(np.float32)
    desired = observation['desired_goal'][task].astype(np.float32)
    obs = observation['observation'].astype(np.float32)

    flat_obs = np.concatenate([achieved, desired, obs], dtype=np.float32)
    return flat_obs

In [59]:
flat_dim = 73
action_space_size = granularity*9+1
obs_low = np.full((flat_dim,), -1e10, dtype=np.float32)
obs_high = np.full((flat_dim,), 1e10, dtype=np.float32)

class FlattenDictWrapper(gym.Wrapper):    
    def __init__(self, env):
        super().__init__(env)
        self.keys = env.observation_space.spaces.keys()
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, shape=(flat_dim,), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(n=action_space_size)

    def observation(self, observation):
        return flatten_observation(observation)
    
    def action(self, action):
        return transform_action_to_int(action)
    
    def step(self, action):
        transformed_action = transform_action_to_int(action)
        obs, reward, terminated, truncated, info = self.env.step(transformed_action)
        obs = flatten_observation(obs)
        return obs, reward, terminated, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return flatten_observation(obs)
    
    
def make_env():
    env = gym.make(env_id, render_mode=None, tasks_to_complete=[task])  # Or your actual task
    env = FlattenDictWrapper(env)
    return env


In [60]:
n_training_envs = 64
env = DummyVecEnv([make_env]*n_training_envs)
eval_env = DummyVecEnv([make_env])

In [80]:
max_timesteps = 250000
exploration_fraction=0.95
run_name = f"dqn_{granularity}p_{max_timesteps}_{int(exploration_fraction*100)}_"+task

In [81]:
model = DQN("MlpPolicy", env, device=device, verbose=1, exploration_fraction=exploration_fraction)

Using cuda:0 device


In [62]:
# model = DQN.load("dqn_3_10000_"+task)

In [82]:
eval_log_dir = os.path.join("eval_logs", run_name)
eval_callback = EvalCallback(eval_env, best_model_save_path=eval_log_dir,
                              log_path=eval_log_dir, eval_freq=max(500 // n_training_envs, 1),
                              n_eval_episodes=5, deterministic=True,
                              render=False)

In [83]:

model.learn(total_timesteps=max_timesteps, callback=eval_callback)

Eval num_timesteps=448, episode_reward=0.00 +/- 0.00
Episode length: 280.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 280      |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.998    |
| time/               |          |
|    total_timesteps  | 448      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00986  |
|    n_updates        | 1        |
----------------------------------
New best mean reward!
Eval num_timesteps=896, episode_reward=0.00 +/- 0.00
Episode length: 280.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 280      |
|    mean_reward      | 0        |
| rollout/            |          |
|    exploration_rate | 0.997    |
| time/               |          |
|    total_timesteps  | 896      |
| train/              |          |
|    learning_rate    | 0.0001   |
|  

KeyboardInterrupt: 

In [66]:
model.save(run_name)

In [67]:
for i in range(10):
	env_eval = make_env()
	obs, _ = env_eval.reset()
	done = False
	ep_reward = 0

	while not done:
		action, _ = model.predict(obs, deterministic=True)
		obs, reward, terminated, truncated, _ = env_eval.step(transform_action_from_int(action))
		obs = flatten_observation(obs)
		done = terminated or truncated
		ep_reward += reward
	print(f"Episode reward: {ep_reward}")

Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
Episode reward: 0.0
